In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
# Load the ferry trips data
ferry_data = pd.read_csv('data/ferry_trips_data_large.csv')

# Display the first few rows to understand the structure
ferry_data.head()
# Summary of the dataset
ferry_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69358 entries, 0 to 69357
Data columns (total 38 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   route_id                                       69358 non-null  int64  
 1   route_name                                     69358 non-null  object 
 2   ferry_name                                     69358 non-null  object 
 3   ferry_id                                       69358 non-null  int64  
 4   terminal_departure                             69358 non-null  object 
 5   terminal_arrival                               69358 non-null  object 
 6   time_departure                                 69358 non-null  object 
 7   cars_outbound                                  69358 non-null  int64  
 8   trucks_outbound                                69358 non-null  int64  
 9   trucks_with_trailer_outbound                   693

## Q1 What is the total fuel cost for ferry Jupiter in January 2024 assuming that the price per liter is 20 sek?

In [19]:
# Convert 'time_departure' to datetime objects for filtering
ferry_data['time_departure'] = pd.to_datetime(ferry_data['time_departure'])

# Filter data for ferry 'Jupiter' and January 2024
jupiter_jan_data = ferry_data[
    (ferry_data['ferry_name'] == 'Jupiter') &
    (ferry_data['time_departure'].dt.month == 1) &
    (ferry_data['time_departure'].dt.year == 2024)
]

# Calculate total fuel consumption (handling potential NaNs by filling with 0)
total_fuel_consumption = (
    jupiter_jan_data['fuelcons_outbound_l'].fillna(0).sum() +
    jupiter_jan_data['fuelcons_inbound_l'].fillna(0).sum()
)

# Assuming a fuel cost per liter (you might need to adjust this)
# Let's assume a fuel cost of 10 SEK per liter for example purposes
fuel_cost_per_liter = 20

# Calculate the total fuel cost
total_fuel_cost = total_fuel_consumption * fuel_cost_per_liter

print(f"Total fuel consumption for ferry Jupiter in January 2024: {total_fuel_consumption:.2f} liters")
print(f"Total fuel cost for ferry Jupiter in January 2024: {total_fuel_cost:.2f} SEK")

Total fuel consumption for ferry Jupiter in January 2024: 12709.34 liters
Total fuel cost for ferry Jupiter in January 2024: 254186.70 SEK


## Q2 What is the average speed of ferry Jupiter? (in km/h)


In [20]:
# Convert time columns to datetime objects
ferry_data['start_time_outbound'] = pd.to_datetime(ferry_data['start_time_outbound'])
ferry_data['end_time_outbound'] = pd.to_datetime(ferry_data['end_time_outbound'])
ferry_data['start_time_inbound'] = pd.to_datetime(ferry_data['start_time_inbound'])
ferry_data['end_time_inbound'] = pd.to_datetime(ferry_data['end_time_inbound'])

# Filter data for ferry 'Jupiter'
jupiter_data = ferry_data[ferry_data['ferry_name'] == 'Jupiter'].copy()

# Function to calculate trip duration in hours, handling potential NaNs
def calculate_duration(start_time, end_time):
    if pd.notna(start_time) and pd.notna(end_time):
        duration = (end_time - start_time).total_seconds() / 3600
        return duration
    return None

# Apply the function to calculate outbound and inbound durations
jupiter_data['duration_outbound'] = jupiter_data.apply(
    lambda row: calculate_duration(row['start_time_outbound'], row['end_time_outbound']), axis=1
)
jupiter_data['duration_inbound'] = jupiter_data.apply(
    lambda row: calculate_duration(row['start_time_inbound'], row['end_time_inbound']), axis=1
)

# Calculate average speed for outbound trips (distance in nautical miles, duration in hours)
jupiter_data['speed_outbound_knots'] = jupiter_data.apply(
    lambda row: row['distance_outbound_nm'] / row['duration_outbound']
    if row['duration_outbound'] else None,
    axis=1
)

# Calculate average speed for inbound trips
jupiter_data['speed_inbound_knots'] = jupiter_data.apply(
    lambda row: row['distance_inbound_nm'] / row['duration_inbound']
    if row['duration_inbound'] else None,
    axis=1
)

# Calculate the average of all valid speed values
valid_speeds = pd.concat([jupiter_data['speed_outbound_knots'].dropna(), jupiter_data['speed_inbound_knots'].dropna()])

if not valid_speeds.empty:
    average_speed_knots = valid_speeds.mean()
    print(f"The average speed of ferry Jupiter is: {average_speed_knots:.2f} knots")

    # Convert knots to km/h and mph for better understanding
    average_speed_kmh = average_speed_knots * 1.852
    average_speed_mph = average_speed_knots * 1.15078
    print(f"Average speed in km/h: {average_speed_kmh:.2f} km/h")
    print(f"Average speed in mph: {average_speed_mph:.2f} mph")
else:
    print("Could not calculate the average speed for ferry Jupiter due to missing distance or time information.")

The average speed of ferry Jupiter is: 6.23 knots
Average speed in km/h: 11.55 km/h
Average speed in mph: 7.17 mph


## Q3 How many vehicles left at the terminal are there in total? 

In [27]:
# Total vehicles left at the terminal
vehicles_left_total = ferry_data['vehicles_left_at_terminal_outbound'].sum() + ferry_data['vehicles_left_at_terminal_inbound'].sum()

# Average vehicles left at terminal per trip
vehicles_left_avg = ferry_data[['vehicles_left_at_terminal_outbound', 'vehicles_left_at_terminal_inbound']].mean()

print(f"Total vehicles left: {vehicles_left_total}")
print(vehicles_left_avg)

Total vehicles left: 44748
vehicles_left_at_terminal_outbound    0.318175
vehicles_left_at_terminal_inbound     0.326999
dtype: float64


## Q4 How many vehicles on average are left at the terminal per trip for ferry Jupiter during summer months?

In [29]:
# Convert 'time_departure' to datetime objects for filtering
ferry_data['time_departure'] = pd.to_datetime(ferry_data['time_departure'], errors='coerce')

# Filter data for ferry 'Jupiter'
jupiter_data = ferry_data[ferry_data['ferry_name'] == 'Jupiter'].copy()

# Define summer months (June, July, August)
summer_months = [6, 7, 8]

# Filter Jupiter data for summer months, handling potential NaT values
jupiter_summer_data = jupiter_data[
    jupiter_data['time_departure'].notna() &
    jupiter_data['time_departure'].dt.month.isin(summer_months)
]

# Calculate the total vehicles left at the terminal during summer for Jupiter
# Assuming the columns for vehicles left are:
vehicles_left_cols = ['cars_outbound', 'trucks_outbound', 'trucks_with_trailer_outbound',
                      'cars_inbound', 'trucks_inbound', 'trucks_with_trailer_inbound']

total_left_summer_jupiter = jupiter_summer_data[vehicles_left_cols].fillna(0).sum().sum()

# Calculate the total number of trips made by Jupiter during the summer
total_trips_summer_jupiter = len(jupiter_summer_data)

# Calculate the average amount of vehicles left per trip for Jupiter during summer
average_left_summer_jupiter = total_left_summer_jupiter / total_trips_summer_jupiter if total_trips_summer_jupiter > 0 else 0

print(f"Total vehicles left at the terminal for ferry Jupiter during summer: {total_left_summer_jupiter}")
print(f"Average amount of vehicles left at the terminal per trip for ferry Jupiter during summer: {average_left_summer_jupiter:.2f}")


Total vehicles left at the terminal for ferry Jupiter during summer: 199263
Average amount of vehicles left at the terminal per trip for ferry Jupiter during summer: 43.21
